•
Tu vas effectuer une classification supervisée sur la colonne "survived", en effectuant un traintestsplit puis un classifieur de ton choix (knn, decision tree, régression logistique, etc...). Tu dois faire apparaitre clairement l'accuracy score du jeu d'entrainement et celui du jeu de test.

•
Avec ce même classifier, tu vas lancer une crossvalidation avec un découpage en 6 parties (k-fold = 6), et avec découpage aléatoire. Le scoring doit être l'accuracy score. La CrossValidation renforce-t-elle votre confiance dans cette prédiction ?

# Import des modules

In [0]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold

# Import du dataset

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
# Download du fichier
id = '1CmJRYacimunpj7EUAic4LZL-dmZkUXjF'
downloaded = drive.CreateFile({'id': '1CmJRYacimunpj7EUAic4LZL-dmZkUXjF'})
downloaded.GetContentFile('titanic.csv')

In [0]:
df_titanic = pd.read_csv('titanic.csv')
df_titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


# Classification supervisée

In [0]:
# création d'une colone Sex_male (booleen) à partir de la colonne sex
le = LabelEncoder()
le.fit(df_titanic['Sex'])
Sex_male = le.transform(df_titanic['Sex'])
df_titanic['Sex_male'] = Sex_male

In [0]:
#définition des variables 
X = df_titanic[['Pclass', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare', 'Sex_male']]
y = df_titanic['Survived']

In [0]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## KNN

In [0]:
# definition du modèle
n_neighbors = int(df_titanic.shape[0]**0.5)
Survivor = KNeighborsClassifier(n_neighbors=n_neighbors)
Survivor.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=29, p=2,
                     weights='uniform')

In [0]:
# prédictions
y_pred_test = Survivor.predict(X_test)
y_pred_train = Survivor.predict(X_train)

In [0]:
#accuracy score
accuracy_score_test = accuracy_score(y_test, y_pred_test)
accuracy_score_train = accuracy_score(y_train, y_pred_train)

print("L'accuracy score du jeu de test est de {}".format(accuracy_score_test))
print("L'accuracy score du jeu d'entraînement est de {}".format(accuracy_score_train))

L'accuracy score du jeu de test est de 0.6981981981981982
L'accuracy score du jeu d'entraînement est de 0.7308270676691729


# Cross validation

In [0]:
# 6 parties, découpage aléatoire, accuracy score

cross_val_score(Survivor, X, y, cv=StratifiedKFold(n_splits=6, random_state = 42, shuffle=True), scoring=make_scorer(accuracy_score))

array([0.69594595, 0.7027027 , 0.75      , 0.69594595, 0.71621622,
       0.68027211])

Les résultats de la validation croisée confirment les scores du modèle, autour de 70% (+/- 5%)